In [3]:
import pandas as pd

In [4]:
steam_games = pd.read_json('../data/original/steam_games.json.gz',compression='gzip',lines=True)

In [5]:
# import os
# os.path.join('..','data','original','users_items.json.gz')

In [6]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [7]:
#calculo el porcentaje de valores nulos en cada columna (son muchas, mas del 70%)
100*steam_games.isna().sum()/len(steam_games)

publisher       80.004982
genres          76.045498
app_name        73.321433
title           75.021794
url             73.319773
release_date    75.035909
tags            73.455104
reviews_url     73.321433
specs           73.876043
price           74.463033
early_access    73.319773
id              73.321433
developer       76.058782
dtype: float64

In [8]:
#Borro todas las filas que estan completamente Nulas
steam_games = steam_games.dropna(how='all')

In [9]:
#vuelvo a calcular el porcentaje de valores nulos en cada columna
100*steam_games.isna().sum()/len(steam_games)

publisher       25.056792
genres          10.216275
app_name         0.006224
title            6.379337
url              0.000000
release_date     6.432239
tags             0.507235
reviews_url      0.006224
specs            2.084954
price            4.285047
early_access     0.000000
id               0.006224
developer       10.266065
dtype: float64

In [12]:
#cantidad de nulos por columna
steam_games.isna().sum()

publisher       8052
genres           139
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [11]:
# Completar los generos con las etiquetas de los Tags
steam_games['genres'] = steam_games['tags'].fillna(steam_games['genres'])

In [13]:
steam_games.sample(10)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
97285,2xMilk,"[Early Access, Action, Indie, Racing, Parkour,...",VThree,VThree,http://store.steampowered.com/app/567320/VThree/,2017-02-28,"[Early Access, Action, Indie, Racing, Parkour,...",http://steamcommunity.com/app/567320/reviews/?...,"[Single-player, Steam Achievements, Steam Work...",7.99,1.0,567320.0,2xMilk
113261,None,"[Adventure, RPG]",Epic Battle Fantasy 4 - Soundtrack,Epic Battle Fantasy 4 - Soundtrack,http://store.steampowered.com/app/419290/Epic_...,2015-11-16,"[Adventure, RPG]",http://steamcommunity.com/app/419290/reviews/?...,"[Single-player, Downloadable Content]",None,0.0,419290.0,Phyrnna
96452,None,"[Movie, Sci-fi, Action, Classic]",Flash Gordon: Spaceship to the Unknown,Flash Gordon: Spaceship to the Unknown,http://store.steampowered.com/app/565560/Flash...,2016-12-06,"[Movie, Sci-fi, Action, Classic]",http://steamcommunity.com/app/565560/reviews/?...,[Captions available],2.99,0.0,565560.0,None
93050,Dovetail Games,[Simulation],TS Marketplace: Loadhaul CEA Covered Hopper Wa...,TS Marketplace: Loadhaul CEA Covered Hopper Wa...,http://store.steampowered.com/app/326014/TS_Ma...,2015-11-04,[Simulation],http://steamcommunity.com/app/326014/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",4.99,0.0,326014.0,Fastline Simulation
101848,Codevictus Inc,"[Early Access, Indie, Sports, Action]",Jetball,Jetball,http://store.steampowered.com/app/666590/Jetball/,2017-12-06,"[Early Access, Indie, Sports, Action]",http://steamcommunity.com/app/666590/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",9.99,1.0,666590.0,Codevictus Inc.
93647,Targem Games,"[Casual, Action, Indie, Rhythm, Music]",Dance Magic,Dance Magic,http://store.steampowered.com/app/394260/Dance...,2016-02-15,"[Casual, Action, Indie, Rhythm, Music]",http://steamcommunity.com/app/394260/reviews/?...,"[Single-player, Multi-player, Steam Achievemen...",9.99,0.0,394260.0,Targem Games
110588,505 Games,"[Casual, Strategy, Indie]",SUPERFIGHT - The Digital Deck,SUPERFIGHT - The Digital Deck,http://store.steampowered.com/app/460840/SUPER...,2016-07-21,"[Casual, Strategy, Indie]",http://steamcommunity.com/app/460840/reviews/?...,"[Single-player, Multi-player, Downloadable Con...",4.99,0.0,460840.0,Pipeworks Studio
93610,GSProductions Ltd.,"[RPG, Simulation, Management, Strategy, Indie,...",RPG Tycoon,RPG Tycoon,http://store.steampowered.com/app/314240/RPG_T...,2016-02-12,"[RPG, Simulation, Management, Strategy, Indie,...",http://steamcommunity.com/app/314240/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",9.99,0.0,314240.0,Skatanic Studios
104334,Eagle Dynamics SA,[Simulation],F-15C: Red Flag Campaign,F-15C: Red Flag Campaign,http://store.steampowered.com/app/411900/F15C_...,2017-08-30,[Simulation],http://steamcommunity.com/app/411900/reviews/?...,"[Single-player, Downloadable Content, Captions...",9.99,0.0,411900.0,Eagle Dynamics SA
98814,Ripknot Systems,"[Action, Indie, Casual]",Grav Blazer,Grav Blazer,http://store.steampowered.com/app/657590/Grav_...,2017-07-17,"[Action, Indie, Casual]",http://steamcommunity.com/app/657590/reviews/?...,"[Single-player, Steam Achievements]",0.99,0.0,657590.0,Ripknot Systems
